In [1]:
# Import pandas
import pandas as pd
# Import the required dependencies from sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import gradio as gr
import pandas as pd
from finvizfinance.quote import finvizfinance

In [2]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import gradio as gr
import pandas as pd


# Download VADER lexicon for sentiment analysis
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

# Define a function to calculate sentiment
def extract_score(text):
    score = sid.polarity_scores(text)
    compound = score['compound']
    sentiment = 'neutral'
    if compound >= 0.05:
        sentiment = "positive"
    elif compound <= -0.05:
        sentiment = "negative"
    return sentiment

# Function to process the stock ticker and generate synopsis
def process_ticker(ticker):
    ticker = ticker.upper()
    stock = finvizfinance(ticker)
    news_df = stock.ticker_news()

    # Add predicted sentiment column
    news_df["Predicted Sentiment"] = news_df["Title"].apply(extract_score)
    
    # Generate a synopsis summary of sentiment
    sentiment_counts = news_df["Predicted Sentiment"].value_counts()
    synopsis = sentiment_counts.to_string()  
    
    return news_df[["Title", "Predicted Sentiment"]], synopsis  

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\dclar\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
# Function to clear all outputs
def clear_data():
    # Return empty inputs and an empty DataFrame with correct headers
     return "", pd.DataFrame(columns=["Headline", "Predicted Sentiment"]), ""

# Gradio Interface
with gr.Blocks(theme=gr.themes.Glass(primary_hue="blue")) as app:
    gr.Markdown("# 📈 Headline News Sentiment Analyzer")
    gr.Markdown(
        """
        Analyze the sentiment of recent news headlines for a given stock ticker.
        Just enter a valid ticker symbol (e.g., `AAPL`, `GOOGL`, `TSLA`) to see the predicted sentiment of recent news articles.
        """
    )
    with gr.Row():
        ticker_input = gr.Textbox(
            label="**Enter Stock Ticker**",  # Bold label for clarity
            placeholder="e.g., AAPL",
            lines=1,
        )
    
    with gr.Row():
        analyze_button = gr.Button("Analyze Sentiment")
        clear_button = gr.Button("Clear")

    sentiment_table = gr.DataFrame(
        headers=["Headline", "Predicted Sentiment"],
        label="News Sentiment Analysis Results",
        datatype=["str", "str"],
        wrap=True,  # Enable text wrapping for columns
    )
    synopsis_box = gr.Textbox(
        label="Sentiment Synopsis",
        placeholder="Summary of sentiment counts will appear here.",
        lines=3,
        interactive=False,  # Read-only
    )

    # Button actions
    analyze_button.click(
        process_ticker, 
        inputs=[ticker_input], 
        outputs=[sentiment_table, synopsis_box]
    )
    clear_button.click(
        clear_data, 
        inputs=[], 
        outputs=[ticker_input, sentiment_table, synopsis_box]
    )

# Custom CSS for layout and styling
app.css = """
/* Add a thick black border around the entire Gradio form */
.gradio-container {
    border: 4px solid black !important;
    padding: 20px;
    margin: 0 auto;
    max-width: 800px;  /* Center and limit the form width */
}

/* Adjust font size and weight for the textbox label */
.gr-textbox .gr-label {
    font-size: 18px !important;
    font-weight: bold !important;
    color: #000 !important;
}

/* Adjust the column widths in the DataFrame */
.gr-dataframe {
    margin: 0 auto; /* Center the DataFrame */
    width: 80%; /* Set the width of the DataFrame */
}

.gr-dataframe th:nth-child(1), /* Headline column */
.gr-dataframe td:nth-child(1) {
    max-width: 450px; /* Wider width for Headline */
    white-space: normal !important; /* Enable text wrapping */
}

.gr-dataframe th:nth-child(2), /* Predicted Sentiment column */
.gr-dataframe td:nth-child(2) {
    max-width: 150px; /* Narrower width for Predicted Sentiment */
    text-align: center;
}

/* Style for the synopsis box */
.gr-textbox {
    width: 3in; /* Set width to 4 inches */
    margin: 20px auto; /* Center align */
    border: 1px solid black; /* Thinner black border around synopsis */
    padding: 10px; /* Add padding for better readability */
    background-color: #f9f9f9; /* Light gray background */
}
"""

# Launch the app
app.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
